# Notebook that handles fine-tuning from start to finish

In [2]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-15w0hyf3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-15w0hyf3
  Resolved https://github.com/huggingface/transformers.git to commit 1982dd3b15867c46e1c20645901b0de469fd935f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached regex-2023.8.8-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached safetensors-0.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7522589 sha256=52234025dd643f81

In [3]:
!pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" --upgrade

  Using cached sagemaker-2.178.0-py2.py3-none-any.whl
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached datasets-2.10.1-py3-none-any.whl (469 kB)
  Using cached aiohttp-3.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached xxhash-3.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (193 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (738 kB)
  Using cached boto3-1.28.30-py3-none-any.whl (135 kB)
  Using cached jsonschema-4.19.0-py3-none-any.whl (83 kB)
  Using cached platformdirs-3.10.0-py3-none-any.whl (17 kB)
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
  Using cached botocore-1.31.30-py3-none-any.whl (11.1 MB)
  Using cached frozenlist-1.4.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (228 kB)
  Using ca

In [4]:
!pip install accelerate==0.20.3

  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.16.0
    Uninstalling accelerate-0.16.0:
      Successfully uninstalled accelerate-0.16.0

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [135]:
import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import json

#from models.EDdisposition import EDdispositionClassifier

In [136]:
# reads out files

import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket('chianglab-dataderivatives')

folders = set()

for obj in bucket.objects.all():
    prefix, delimiter, _ = obj.key.rpartition('/')
    if prefix:
        folders.add(prefix + '/')

print('Folders:')
for folder in folders:
    print(folder)
    


Folders:
mimic-iv-2.2/
mimic-iv-clinical-database-demo-2.2/
mimic-iv-ed-2.2/
mimic-iv-ed-demo-2.2/


In [137]:
bucket = 'chianglab-dataderivatives'
subfolder = 'mimic-iv-ed-demo-2.2/'
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

file_list = []

for f in contents:
    print(f['Key'])
    file_list.append(f['Key'][36:])

print(file_list)

mimic-iv-ed-demo-2.2/
mimic-iv-ed-demo-2.2/text_repr.json
['', '']


### Load in Data and reformat 

In [138]:
s3 = boto3.resource('s3')
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/admissions_text.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

df = pd.DataFrame(list(json_content.items()), columns = ['ID', 'ad_headline'])

In [139]:
# add codes 

bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/codes_text.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df['codes_headline'] = df['ID'].map(json_content)

# add meds
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/meds_text.json"

content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df['meds_headline'] = df['ID'].map(json_content)

# add labs
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/labs_text.json"

content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
# need to change float IDs to int but then back to a string to map
res = dict()
for key in json_content:
    res[str(int(float(key)))] = json_content[key]
    
df['labs_headline'] = df['ID'].map(res)

### Data we will be working with

In [140]:
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-demo-2.2/text_repr.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T
print("length of dataframe: "+ str(len(df)))
df.head(5)
# df['codes_headline'] = df['ID'].map(json_content)


length of dataframe: 210


,arrival,eddischarge,admission,discharge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",The ED disposition was admitted at 2125-03-19 ...,The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",The ED disposition was admitted at 2154-02-05 ...,The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,NaN,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",The ED disposition was home at 2154-08-03 22:2...,The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",The ED disposition was admitted at 2153-02-13 ...,The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",The ED disposition was admitted at 2137-06-12 ...,The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...


In [141]:
df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df['medrecon'] = df['medrecon'].fillna("The patient was previously not taking any medications.")
df['pyxis'] = df['pyxis'].fillna("The patient did not receive any medications.")
df['vitals'] = df['vitals'].fillna("The patient had no vitals recorded")
df['codes'] = df['codes'].fillna("The patient received no diagnostic codes")
df = df.drop("admission",axis=1)
df = df.drop("discharge",axis=1)
df

# remove admission and discharge columns 

,arrival,eddischarge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",1,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",1,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient did not receive any medications.,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",0,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",1,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",1,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...
...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...",1,"At triage: temperature was not recorded, pulse...",The patient was previously not taking any medi...,The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...
31628990,"Patient 10009049, a 56 year old white male, ar...",1,"At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...
32405286,"Patient 10004457, a 65 year old white male, ar...",1,"At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...
34391979,"Patient 10004720, a 61 year old white male, ar...",1,"At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...


In [143]:
df['general'] = df['arrival'] + df['triage'] + df['codes'] + df['pyxis'] 
df = df.drop('arrival',axis=1)
df = df.drop('triage',axis=1)
df = df.drop('codes',axis=1)
df = df.drop('pyxis',axis=1)
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df

,medrecon,vitals,general,eddischarge
37887480,The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,"Patient 10014729, a 21 year old white - other ...",1
34176810,The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,"Patient 10018328, a 83 year old white female, ...",1
32103106,The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,"Patient 10018328, a 83 year old white female, ...",0
38797992,The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,"Patient 10020640, a 91 year old white female, ...",1
33473053,The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,"Patient 10015272, a 78 year old white female, ...",1
...,...,...,...,...
30272878,The patient was previously not taking any medi...,The patient had the following vitals: At 2131-...,"Patient 10038999, a 45 year old white male, ar...",1
31628990,The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,"Patient 10009049, a 56 year old white male, ar...",1
32405286,The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,"Patient 10004457, a 65 year old white male, ar...",1
34391979,The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,"Patient 10004720, a 61 year old white male, ar...",1


In [144]:
# split dataframe here
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=None):
    np.random.seed(seed)
    df = df.reset_index()
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    train = train.set_index('index')
    validate = validate.set_index('index')
    test = test.set_index('index')
    return train, validate, test

In [145]:
train, validate, test = train_validate_test_split(df)
print(len(train), len(validate), len(test))

168 21 21


In [146]:
display(train.head(2))
display(validate.head(2))
display(test.head(2))

,medrecon,vitals,general,eddischarge
index,,,,
31023359,The patient was previously taking the followin...,The patient had the following vitals: At 2149-...,"Patient 10005866, a 57 year old portuguese mal...",0
38213541,The patient was previously not taking any medi...,The patient had the following vitals: At 2116-...,"Patient 10017492, a 84 year old patient declin...",1


,medrecon,vitals,general,eddischarge
index,,,,
34176810,The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,"Patient 10018328, a 83 year old white female, ...",1
37330786,The patient was previously taking the followin...,The patient had the following vitals: At 2115-...,"Patient 10035631, a 63 year old white male, ar...",1


,medrecon,vitals,general,eddischarge
index,,,,
35933027,The patient was previously taking the followin...,The patient had the following vitals: At 2176-...,"Patient 10012853, a 91 year old black/african ...",0
32287800,The patient was previously taking the followin...,The patient had the following vitals: At 2147-...,"Patient 10014354, a 60 year old white male, ar...",1


In [148]:
def cut(df, type):
    col_names = df.columns.drop("eddischarge")
    l = []
    for i in col_names:
        temp = df[[i, 'eddischarge']].reset_index()
        temp = temp.sort_values(by=['index']).reset_index() # we sort the patient ID numerically before dropping it to preserve order in encoding
        temp = temp.drop(columns=["index", "level_0"])
        temp = temp.rename(columns={i: "headline", "eddischarge": "label"})
        l.append(temp)
        print("\""+i+ "\" Dataframe:", type, "set")
    return l

In [149]:
l1 = cut(train, "train")
l2 = cut(validate, "validate")
l3 = cut (test, "test")

"medrecon" Dataframe: train set
"vitals" Dataframe: train set
"general" Dataframe: train set
"medrecon" Dataframe: validate set
"vitals" Dataframe: validate set
"general" Dataframe: validate set
"medrecon" Dataframe: test set
"vitals" Dataframe: test set
"general" Dataframe: test set


In [150]:
# Indices for sub-dataframes, that will each individually be tokenized
# 0 - arrival
# 1 - admission
# 2 - discharge
# 3 - triage
# 4 - medrecon
# 5 - vitals
# 6 - pyxis
# 7 - codes
class preprocess():

    def tokenize(self,batch):
      return tokenizer(batch["headline"], truncation=True, max_length=512)

    def convert(self, l):
        """
        Run this method
        """
        medrecon_hf=Dataset.from_pandas(l[0])
        vitals_hf=Dataset.from_pandas(l[1])
        general_hf=Dataset.from_pandas(l[2])

        tokenized_medrecon = medrecon_hf.map(tokenize, batched=True)
        tokenized_vitals = vitals_hf.map(tokenize, batched=True)
        tokenized_general = general_hf.map(tokenize, batched=True)
        
        dataset_cc = concatenate_datasets([tokenized_medrecon, tokenized_vitals, tokenized_general])
        print("Specs:")
        print(dataset_cc)
        return dataset_cc

In [151]:
# grabs a tokenizer from the distilbert-base-uncased model
model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.model_max_len=512

# calls methods and tokenizes text
processor = preprocess()
train = processor.convert(l1)
validate = processor.convert(l2)
test = processor.convert(l3)



Map:   0%|          | 0/168 [00:00<?, ? examples/s]
                                                   
Map:   0%|          | 0/168 [00:00<?, ? examples/s]
                                                   
Map:   0%|          | 0/168 [00:00<?, ? examples/s]
                                                   

Specs:
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 504
})



Map:   0%|          | 0/21 [00:00<?, ? examples/s]
                                                  
Map:   0%|          | 0/21 [00:00<?, ? examples/s]
                                                  
Map:   0%|          | 0/21 [00:00<?, ? examples/s]
                                                  

Specs:
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 63
})



Map:   0%|          | 0/21 [00:00<?, ? examples/s]
                                                  
Map:   0%|          | 0/21 [00:00<?, ? examples/s]
                                                  
Map:   0%|          | 0/21 [00:00<?, ? examples/s]
                                                  

Specs:
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 63
})


In [153]:
train.set_format('torch', columns=["input_ids", "attention_mask", "label"] )
validate.set_format('torch', columns=["input_ids", "attention_mask", "label"] )
test.set_format('torch', columns=["input_ids", "attention_mask", "label"] )
print(train)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 504
})


In [155]:
dataset_cc = DatasetDict({
    'train': train,
    'test': test,
    'valid': validate})


In [156]:
dataset_cc

DatasetDict({
    train: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 504
    })
    test: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 63
    })
    valid: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 63
    })
})

In [157]:
class EDdispositionClassifier(nn.Module):
    """
    A task-specific custom transformer model for predicting ED Disposition. 
    This model loads a pre-trained transformer model and adds a new dropout 
    and linear layer at the end for fine-tuning and prediction on specific tasks.
    """
    def __init__(self, checkpoint, num_labels ):
        """
        Args:
            checkpoint (str): The name of the pre-trained model or path to the model weights.
            num_labels (int): The number of output labels in the final classification layer.
        """
        super(EDdispositionClassifier, self).__init__()
        self.num_labels = num_labels # number of labels for classifier
        
        # checkpoint is the model name 
        self.model = model = AutoModel.from_pretrained(checkpoint, config = AutoConfig.from_pretrained(checkpoint, 
                                                                                                       output_attention = True, 
                                                                                                       output_hidden_state = True ) )
        # New Layer
        self.dropout = nn.Dropout(0.1) # to prevent overfittting
        self.classifier = nn.Linear(768, num_labels) #FC Layer - takes in a 768 token vector and is a Linear classifier with n labels
        
    def forward(self, input_ids = None, attention_mask=None, labels = None ):
        """
        Forward pass for the model.
        
        Args:
            input_ids (torch.Tensor, optional): Tensor of input IDs. Defaults to None.
            attention_mask (torch.Tensor, optional): Tensor for attention masks. Defaults to None.
            labels (torch.Tensor, optional): Tensor for labels. Defaults to None.
            
        Returns:
            TokenClassifierOutput: A named tuple with the following fields:
            - loss (torch.FloatTensor of shape (1,), optional, returned when label_ids is provided) – Classification loss.
            - logits (torch.FloatTensor of shape (batch_size, num_labels)) – Classification scores before SoftMax.
            - hidden_states (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True) – Tuple of torch.FloatTensor (one for the output of the embeddings + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).
            - attentions (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) – Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).
        """
        # calls on the Automodel to deploy correct model - in our case distilled-bert-uncased
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask  )
        
        # retrieves the last hidden state
        last_hidden_state = outputs[0]
        
        # include dropout from constructor to feed forward network
        sequence_outputs = self.dropout(last_hidden_state)
        
        # finally add linear layer from input
        logits = self.classifier(sequence_outputs[:, 0, : ].view(-1, 768 ))
        
        # calculates loss 
        loss = None
        if labels is not None:
            loss_func = nn.CrossEntropyLoss() # Change this if it becomes more than binary classification
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
            
            # TokenClassifierOutput - returns predicted label
            return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)
        

In [158]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset_cc['train'], shuffle = True, batch_size = 8, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    dataset_cc['valid'], shuffle = True, collate_fn = data_collator
)


In [159]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_task_specific = EDdispositionClassifier(checkpoint=model, num_labels=2).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [169]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model_task_specific.parameters(), lr = 5e-5 )

num_epoch = 5

num_training_steps = num_epoch * len(train_dataloader)
print(len(train_dataloader))

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

63


In [165]:
from datasets import load_metric
metric = load_metric("f1", "precision")


In [166]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader) ))


for epoch in range(num_epoch):
    model_task_specific.train()
    for batch in train_dataloader:
        batch = { k: v.to(device) for k, v in batch.items() }
        outputs = model_task_specific(**batch)
        loss = outputs.loss
        loss.backward() # computes gradients
        
        optimizer.step() # updates the weights and biases based on these gradients
        lr_scheduler.step() # updates the weights and biases based on these gradients
        optimizer.zero_grad() # used to clear the gradients of all parameters in a model
        progress_bar_train.update(1)
    
    # run on validation set
    model_task_specific.eval()
    for batch in eval_dataloader:
        batch = { k: v.to(device) for k, v in batch.items() }
        with torch.no_grad(): # a context manager that disables gradient calculation during model inference
            outputs = model_task_specific(**batch)
            
        logits = outputs.logits # calculates the probabilities between the labels
        predictions = torch.argmax(logits, dim = -1 ) # takes the label closest to 1
        metric.add_batch(predictions = predictions, references = batch['labels'] ) 
        progress_bar_eval.update(1)
        
    print(metric.compute()) 


100%|██████████| 504/504 [1:47:35<00:00, 12.81s/it]


510it [1:47:35, 12.66s/it]:00<?, ?it/s]
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.

 20%|██        | 63/315 [02:15<08:56,  2.13s/it]

  0%|          | 1/315 [02:15<11:49:42, 135.61s/it]

  1%|▏         | 4/315 [02:15<2:13:11, 25.70s/it]  

  2%|▏         | 7/315 [02:15<1:01:14, 11.93s/it]

  3%|▎         | 10/315 [02:16<34:26,  6.77s/it] 

  4%|▍         | 14/315 [02:16<18:41,  3.72s/it]

  5%|▌         | 17/315 [02:16<12:32,  2.52s/it]

  6%|▋         | 20/315 [02:16<08:32,  1.74s/it]

  7%|▋         | 23/315 [02:16<05:52,  1.21s/it]

  8%|▊         | 26/315 [02:16<04:05,  1.18it/s]

  9%|▉         | 29/315 [02:16<02:52,  1.66it/s]

 10%|█         | 32/315 [02:16<02:01,  2.32it/s]

 11%|█         | 35/315 [02:17<01:27,  3.19it/s]

 12%|█▏       

{'f1': 0.8}



 23%|██▎       | 72/315 [02:34<06:49,  1.69s/it]

 40%|████      | 126/315 [04:32<07:26,  2.36s/it]

 20%|██        | 64/315 [04:33<57:52, 13.84s/it]

 21%|██        | 66/315 [04:33<44:27, 10.71s/it]

 22%|██▏       | 69/315 [04:33<29:39,  7.23s/it]

 23%|██▎       | 72/315 [04:33<20:02,  4.95s/it]

 23%|██▎       | 74/315 [04:33<15:17,  3.81s/it]

 25%|██▍       | 78/315 [04:33<09:05,  2.30s/it]

 26%|██▌       | 81/315 [04:33<06:20,  1.63s/it]

 27%|██▋       | 84/315 [04:34<04:26,  1.16s/it]

 28%|██▊       | 87/315 [04:34<03:07,  1.22it/s]

 29%|██▊       | 90/315 [04:34<02:13,  1.69it/s]

 30%|██▉       | 93/315 [04:34<01:34,  2.34it/s]

 30%|███       | 96/315 [04:34<01:07,  3.24it/s]

 31%|███▏      | 99/315 [04:34<00:50,  4.31it/s]

 32%|███▏      | 102/315 [04:34<00:37,  5.69it/s]

 33%|███▎      | 105/315 [04:34<00:28,  7.45it/s]

 34%|███▍      | 108/315 [04:35<00:23,  8.82it/s]

 35%|███▌      | 111/315 [04:35<00:18, 10.97it/s]

 36%|███▌      | 114/315 [04:35<00:16, 12.32

{'f1': 0.8484848484848484}



 41%|████▏     | 130/315 [04:44<07:39,  2.48s/it]

 60%|██████    | 189/315 [06:48<04:07,  1.96s/it]

 40%|████      | 127/315 [06:48<52:13, 16.67s/it]

 41%|████      | 129/315 [06:49<38:06, 12.29s/it]

 42%|████▏     | 133/315 [06:49<21:20,  7.04s/it]

 43%|████▎     | 136/315 [06:49<14:25,  4.83s/it]

 44%|████▍     | 139/315 [06:49<09:49,  3.35s/it]

 45%|████▌     | 142/315 [06:49<06:44,  2.34s/it]

 46%|████▌     | 145/315 [06:49<04:39,  1.64s/it]

 47%|████▋     | 149/315 [06:50<02:54,  1.05s/it]

 48%|████▊     | 152/315 [06:50<02:05,  1.30it/s]

 49%|████▉     | 155/315 [06:50<01:29,  1.78it/s]

 50%|█████     | 158/315 [06:50<01:05,  2.41it/s]

 51%|█████     | 160/315 [06:50<00:52,  2.97it/s]

 52%|█████▏    | 163/315 [06:50<00:36,  4.12it/s]

 53%|█████▎    | 166/315 [06:50<00:27,  5.33it/s]

 53%|█████▎    | 168/315 [06:51<00:23,  6.37it/s]

 54%|█████▍    | 171/315 [06:51<00:17,  8.42it/s]

 55%|█████▌    | 174/315 [06:51<00:13, 10.58it/s]

 56%|█████▌    | 177/315 [06:5

{'f1': 0.8484848484848484}



 62%|██████▏   | 195/315 [07:04<04:36,  2.30s/it]

 80%|████████  | 252/315 [09:04<02:23,  2.28s/it]

 60%|██████    | 190/315 [09:04<34:56, 16.77s/it]

 61%|██████    | 191/315 [09:04<29:24, 14.23s/it]

 61%|██████▏   | 193/315 [09:04<20:10,  9.93s/it]

 62%|██████▏   | 196/315 [09:04<11:57,  6.03s/it]

 63%|██████▎   | 199/315 [09:05<07:29,  3.87s/it]

 64%|██████▍   | 202/315 [09:05<04:50,  2.57s/it]

 65%|██████▌   | 205/315 [09:05<03:11,  1.74s/it]

 66%|██████▌   | 208/315 [09:05<02:09,  1.21s/it]

 67%|██████▋   | 211/315 [09:05<01:27,  1.19it/s]

 68%|██████▊   | 214/315 [09:05<01:00,  1.68it/s]

 69%|██████▉   | 217/315 [09:05<00:42,  2.32it/s]

 70%|██████▉   | 220/315 [09:06<00:30,  3.16it/s]

 71%|███████   | 223/315 [09:06<00:21,  4.30it/s]

 72%|███████▏  | 226/315 [09:06<00:15,  5.58it/s]

 73%|███████▎  | 229/315 [09:06<00:11,  7.36it/s]

 74%|███████▍  | 233/315 [09:06<00:08,  9.97it/s]

 75%|███████▍  | 236/315 [09:06<00:06, 11.59it/s]

 76%|███████▌  | 239/315 [09:0

{'f1': 0.8484848484848484}



 83%|████████▎ | 261/315 [09:26<01:49,  2.03s/it]

100%|██████████| 315/315 [11:20<00:00,  2.17s/it]

 80%|████████  | 253/315 [11:20<15:12, 14.72s/it]

 81%|████████▏ | 256/315 [11:20<09:50, 10.01s/it]

 82%|████████▏ | 259/315 [11:21<06:25,  6.88s/it]

 83%|████████▎ | 262/315 [11:21<04:12,  4.77s/it]

 84%|████████▍ | 265/315 [11:21<02:45,  3.32s/it]

 85%|████████▌ | 268/315 [11:21<01:49,  2.32s/it]

 86%|████████▌ | 271/315 [11:21<01:11,  1.63s/it]

 87%|████████▋ | 275/315 [11:21<00:41,  1.05s/it]

 88%|████████▊ | 278/315 [11:21<00:28,  1.31it/s]

 89%|████████▉ | 281/315 [11:21<00:18,  1.80it/s]

 90%|█████████ | 285/315 [11:22<00:11,  2.70it/s]

 91%|█████████▏| 288/315 [11:22<00:07,  3.55it/s]

 92%|█████████▏| 291/315 [11:22<00:05,  4.71it/s]

 93%|█████████▎| 294/315 [11:22<00:03,  5.89it/s]

 94%|█████████▍| 297/315 [11:22<00:02,  7.49it/s]

 95%|█████████▌| 300/315 [11:22<00:01,  9.11it/s]

 96%|█████████▌| 303/315 [11:22<00:01, 10.79it/s]

 97%|█████████▋| 306/315 [11:2

{'f1': 0.8260869565217391}


# Test predictions

In [167]:
model_task_specific.eval()

test_dataloader = DataLoader(
    dataset_cc['test'], batch_size = 32, collate_fn = data_collator
)

for batch in test_dataloader:
    batch = { k: v.to(device) for k, v in batch.items() }
    with torch.no_grad(): # a context manager that disables gradient calculation during model inference
        outputs = model_task_specific(**batch)

    logits = outputs.logits # calculates the probabilities between the labels
    predictions = torch.argmax(logits, dim = -1 ) # takes the label closest to 1
    metric.add_batch(predictions = predictions, references = batch['labels'] )
    print("New Batch")
    print(predictions)
    print(batch['labels'])

print(metric.compute()) 

New Batch
tensor([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 0, 1, 0, 1, 0])
New Batch
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1])
tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1])
{'f1': 0.8316831683168316}




100%|██████████| 315/315 [11:36<00:00, 13.71it/s]